In [34]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from functools import partial, reduce
from itertools import combinations
from scipy.stats import percentileofscore
import affinity as af
import ratlib as rat

%matplotlib inline 

In [35]:
data = pd.read_excel('~/GIT/collinearity/database/sidis/expdata/1008.xlsx')
tab = pd.read_excel('~/GIT/collinearity/notebooks/SIDIS_Proximity/1008_Affinity.xlsx')

In [36]:
class Rfilter(object):

    def __init__(self, hadron=hadron, fudge=[0, 0]):
        #self.M = 0.938
        self.M = 0.94
        self.M2 = self.M**2
        self.set_Mh(hadron)
        

        self.fudge1 = fudge[0]
        self.fudge2 = fudge[1]

        self.MiT2 = 0.5 + 0.3 * fudge[0]
        self.MfT2 = 0.5 + 0.3 * fudge[1]
        self.kT = 0
    
        self.MX = 1.3
        self.Ma = 1.5
        self.Mb = 0.3
        self.deltaM = 0.3
        self.MJ = 0.3
        
        self.MiT = self.MiT2**0.5
        self.MfT = self.MfT2**0.5
        self.kT2 = self.kT**2
        
        # from paper, we need kf2, deltakT2, ki2
        # we need to have a way to vary them!
        self.kf       = 0
        self.ki       = 0
        self.kf2      = self.kf**2
        self.ki2      = self.ki**2
        self.deltakT  = 0
        self.deltakT2 = self.deltakT**2
        
    def set_ki(self,ki):
        self.ki = ki
        self.ki2 = ki**2
        
    def set_kf(self,kf):
        self.kf = kf
        self.kf2 = kf**2
        
    def set_kT(self,kT):
        self.kT = kT
        self.kT2 = kT**2
        
    def set_deltakT(self,deltakT):
        self.deltakT = deltakT
        self.deltakT2 = self.deltakT**2
        
    def set_Mh(self, hadron):
        if hadron == 'pi+':
            self.Mh = 0.135  #  
        if hadron == 'pi-':
            self.Mh = 0.135  #  
        if hadron == 'pi0':
            self.Mh = 0.135  #  
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'k-':
            self.Mh = 0.493
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'h+':
            #self.Mh = 0.135
            self.Mh = 0.14
        if hadron == 'h-':
            self.Mh = 0.135
        self.Mh2 = self.Mh**2
   
    def set_kibT(self,kibT):
        self.kibT = kibT
        self.kibT2 = kibT**2
        
    def set_phi(self,phi):
        self.phi = phi
        
    def get_ki2(self):
        return self.ki2
    
    def get_kf(self):
        return self.kf
    
    def get_W2(self, x, Q2):
        W2 = Q2 * (1. - x) / x + self.M2
        return W2

    def get_MBT(self, PhT):
        MBT = np.sqrt( self.Mh2 + (PhT**2 ))
        return MBT
        
    def get_MiT(self, x, Q2):
        xn = self.get_xn(x, Q2)
#    self.MiT = np.sqrt( (xn*self.kT**2 + xn*self.MX**2 - (1-xn)*xn*self.M2)/(1-xn) )
        self.MiT = np.sqrt((xn * self.kT**2 + xn * (self.Ma + self.Mb / np.sqrt(xn))
                            ** 2 - (1 - xn) * xn * self.M2 + self.deltaM**2 * self.fudge1) / (1 - xn))
        return self.MiT

    def get_MfT(self):
        self.MfT = np.sqrt(self.kT**2 + self.MJ**2 +
                           self.deltaM**2 * self.fudge2)
        return self.MfT

    def get_xn(self, x, Q2):
        return 2 * x / (1 + np.sqrt(1 + 4 * x**2 * self.M2 / Q2))

    def get_yh(self, x, z, Q2, PhT, hadron, sign=-1):
        xn = self.get_xn(x, Q2)
        self.set_Mh(hadron)
        expy = Q2**0.5 * z * (Q2 - xn**2 * self.M2)\
            / (2 * self.M2 * xn**2 * (self.Mh2 + PhT**2)**0.5)\
            + sign * Q2**0.5 / (xn * self.M) * (z**2 * (Q2 - xn**2 * self.M2)**2
                                        / (4 * self.M2 * xn**2 * (self.Mh2 + PhT**2)) - 1)**0.5
        return np.log(expy)


    # zn
    def get_zn(self, x, z, Q2, PhT, hadron):
        xn = self.get_xn(x,Q2)
        self.set_Mh(hadron)
        MBT = self.get_MBT(PhT)
        zn =  xn * z/ (2 * x) * ( 1. + np.sqrt( 1.- 4. * self.M2 * MBT**2 * x**2 /  ( Q2**2 * z**2) ) )
        return zn
        
# rapidity of the target

    def get_yp(self, x, Q2):
        xn = self.get_xn(x, Q2)
        return np.log(np.sqrt(Q2) / (xn * self.M))

    def get_yi(self, Q2):
        return 0.5 * np.log(Q2 / self.MiT**2)

    def get_yf(self, Q2):
        return -0.5 * np.log(Q2 / self.MfT**2)

    def get_MhT(self, PhT):
        return np.sqrt(self.Mh2 + PhT**2)
    
    def get_kibT(self,kibT):
        return self.kibT
    
    def get_Mh(self):
        return self.Mh
    
    def get_Mh2(self):
        return self.Mh2
    
    def get_R(self, x, xi, z, zeta, Q2, PhT, hadron):
        
        zn = self.get_zn(x, z, Q2, PhT, hadron)
        zn2 = zn**2
        xn = self.get_xn(x, Q2)
        znhat = zn/zeta
        xnhat = xn/xi
        qT = -PhT/zn
        
        # from paper..
        Ph_kf = ((((self.Mh2+zn2*qT**2)*znhat)/(2*zn))+(zn*((-znhat*qT+self.deltakT)**2+self.kf2)/(2*znhat))) \
                + zn*qT*(-znhat*qT+self.deltakT)
        Ph_ki = ((((self.Mh2+zn2*qT**2)*(xnhat*(self.ki2 + self.kibT2)))/(2*Q2*zn)) + (zn*Q2)/(2*xnhat))+zn*qT*self.kibT
        
        return np.abs(Ph_kf / Ph_ki)
    def get_R1S(self,x,xi,zeta,Q2,PhT,phi,hadron):
        zn = self.get_zn(x, z, Q2, PhT, hadron)
        zn2 = zn**2
        xn = self.get_xn(x, Q2)
        znhat = zn/zeta
        xnhat = xn/xi
        kibT = self.kibT
        Mh2 = self.Mh2
        deltakT = self.deltakT
        kf = self.kf
        ki = self.ki
        kt = self.kT
        qT = -PhT/zn
        R1 = 1/zeta*(self.Mh2+zeta**2*(self.deltakT2+self.kf2))/((self.Mh2+zn2*qT**2)* \
                 (self.ki2+self.kibT**2)*(xn/(zn*Q2*xi))+ (zn*Q2*xi/xn) + 2*zn*qT*kibT*np.cos(phi))
        return R1
        
        
# R0, Eq. (4.14)...
    def get_R0(self, Q2):
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        R0 = (np.maximum(np.maximum(self.ki**2/Q2,self.kf**2/Q2),self.kT**2/Q2))
        #print(R0)
        return  R0
        
        
# We call R1 in the new paper what was R in the previous...
    def get_R1(self, x, xi, z, zeta, Q2, PhT, hadron):
        Rval.set_Mh(hadron)
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        return  self.get_R(x, xi, z, zeta, Q2, PhT, hadron)

# R2 from Eq. (4.17)        
    def get_R2(self, x, z, zeta, Q2, PhT, hadron):
        self.set_Mh(hadron)
        zn = self.get_zn(x, z, Q2, PhT, hadron)
        znhat = zn/zeta
        qT = -PhT/z
        # from paper..
        return(np.abs( -(1.-znhat) - znhat*qT**2/Q2 - (1.-znhat)*self.kf2/(znhat*Q2) - \
                      self.deltakT2/(znhat*Q2) + 2.*qT*self.deltakT/Q2 ))
    
   


In [26]:
%%time
hadron = 'h+'
fudge = [0,0]
Rval = Rfilter(hadron = hadron,fudge = fudge)
Rval.set_kT(.1)
Rval.set_ki(.1)
Rval.set_kf(.1)
Rval.set_deltakT(.3)
Rval.set_kibT(0.2)
xi = 0.3
zeta = 0.3
x = .2
Q2 = 3**2
z = .3
pT = .1
phi = 0
M = 0.94
M_h = .14

R1 = Rval.get_R1(x, xi, z, zeta, Q2, pT, 'h+')
R2 = Rval.get_R2(x, z, zeta, Q2, pT, 'h+')
R1S2 = Rval.get_R1S(x,xi,zeta,Q2,pT,phi,hadron)
print('Rfilter  ',R1,'',R2)
print('Rfilter R1S ',R1S2)
#print(Rval.get_Mh())
#print(Rval.get_Mh2())
R1S = rat.get_R1( M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,delta_k_t,k_i_t,M_ki,M_kf)
R2S = 0.04860224399921939
print('ratios.py',R1S,R2S)

Rfilter   0.023775178126167786  0.048602633009976616
Rfilter R1S  0.02377517812616779
ratios.py 0.023775081134120253 0.04860224399921939
CPU times: user 2.45 ms, sys: 0 ns, total: 2.45 ms
Wall time: 1.92 ms


In [20]:
# From ratios.py
M= .94
M_h=.14
x_bj = .2
Q = 3
z_h = .3
P_t = 0.1

    #N= 1000
xi = .3 #np.random.uniform(0.3,0.4,N)
zeta = .3 #np.random.uniform(0.3,0.4,N)
delta_k_t = .3 #np.random.uniform(0.3,0.4,N)
k_i_t = .2 #np.random.uniform(0,0.1,N)
M_ki = .1 #np.random.uniform(0,0.1,N)
M_kf = .1 #np.random.uniform(0,0.1,N)

   
    
q_t = -P_t/z_h
eta = .1


In [11]:
# Proximity is defined for a sphere of a selected radius(p) by the following function
# This notebook does not use this function to compare ratios.py with Rfilter

def Proximity(data,Rval,domain1,domain2,domainPhi,p,fudge):
    """
    data    = any Pandas DataFrame should do as long as it contains x,z,Q2, and pT
    domain1 = 
    domain2 = 
    p       = radius of sphere representing the demarcation of the proximity
    hadron  = any hadron found in the data
    fudge   = idk what fudge is so I did the right thing and passed it along 
    """
    
    data.loc[:,'proximity'] = 0.
    
   
    for i in range(1,len(data)):
        
        
        inside = 0.
        outside = 0.
        hadron = data.loc[i,'hadron']
        #domainZeta = np.linspace(data.loc[i,'z']-.1,data.loc[i,'z']+.1,N)
        for ki in domain1:
            Rval.set_ki(ki)
            for kf in domain1:
                Rval.set_kf(kf)
                for kT in domain1:
                    Rval.set_kT(kT)
                    for deltakT in domain2:
                        Rval.set_deltakT(deltakT)
                        for xi in domain2:
                            for zeta in domain2:
                                for phi in domainPhi:
                                    #print(phi)
                                    R0 = Rval.get_R0(data.loc[i,'Q2'])
                            
                                    R1 = Rval.get_R1(data.loc[i,'x'], xi, data.loc[i,'z'], zeta, data.loc[i,'Q2'], \
                                               data.loc[i,'pT'], 'h+')
                                    R2 = Rval.get_R2(data.loc[i,'x'],data.loc[i,'z'], zeta, data.loc[i,'Q2'], \
                                               data.loc[i,'pT'], 'h+')
                            
                                    p2 = R0**2 + R1**2 + R2**2
                            
                                    if p2 < 1:
                                        inside += 1
                                    else:
                                        outside += 1
                                
                            
        data.loc[i,'proximity'] = inside/(inside+outside)
        
        
    return data

In [15]:
%%time
domain1 = np.linspace(0.05,0.8,2,dtype=np.float64)
domain2 = np.linspace(0.3,0.4,2,dtype=np.float64)
domainPhi = np.linspace(0,2*np.pi,2,dtype=np.float64)
p = 1
fudge = [0,0]
data = Proximity(data,Rval,domain1,domain2,domainPhi,p,fudge)

CPU times: user 31 s, sys: 3.42 ms, total: 31 s
Wall time: 31 s


In [10]:
data.to_excel('proximityTest_Nov12.xlsx')